In [7]:
from pathlib import Path

import numpy as np
import pandas as pd
import dask.dataframe as dd
from IPython.display import display
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split


# %matplotlib inline


In [11]:
# read machine_meta table
machine_meta_data_path = 'data/machine_meta.csv'
mm_headers=['machine_id', 'time_stamp', 'failure_domain_1','failure_domain_2','cpu_num','mem_size', 'status']
machine_meta_dataframe = pd.read_csv(machine_meta_data_path, header=None, index_col=False,names=mm_headers,delimiter=',')

# read machine_usage table
machine_usage_data_path = 'data/machine_usage.csv'
mu_headers=['machine_id', 'time_stamp', 'cpu_util_percent', 'mem_util_percent', 'mem_gps', 'mkpi', 'net_in', 'net_out', 'disk_io_percent']
# dfs = []
chunk_size = 10**6
# with pd.read_csv(machine_usage_data_path, chunksize=chunk_size, iterator=True) as reader:
#     for chunk in reader:
#         df = pd.DataFrame(chunk)
#         dfs.append(df)
# machine_usage_dataframe = pd.concat(dfs)
machine_usage_data_frame = dd.read_csv(machine_usage_data_path,names=mu_headers)

# read container_meta table
container_meta_data_path = 'data/container_meta.csv'
cm_headers=['container_id','machine_id','time_stamp','app_du', 'status', 'cpu_request', 'cpu_limit', 'mem_size']
container_meta_dataframe = pd.read_csv(container_meta_data_path, header=None, index_col=False, names=cm_headers,delimiter=',')

cm_stopped_dataframe = container_meta_dataframe[container_meta_dataframe['status'] == 'stopped']
cm_started_dataframe = container_meta_dataframe[container_meta_dataframe['status'] == 'started']

container_meta_dataframe_merged = cm_started_dataframe.merge(cm_stopped_dataframe, on='container_id', how='inner')

# read batch_task table
batch_task_data_path = 'data/batch_task.csv'
bt_headers=['task_name','instance_num', 'job_name', 'task_type', 'status', 'start_time', 'end_time', 'plan_cpu', 'plan_mem']
batch_task_dataframe = dd.read_csv(
    batch_task_data_path, 
    dtype={
        'task_name': 'string',
        'instance_num': 'Int64',
        'job_name': 'string',
        'task_type': 'string',
        'status': 'string',
        'start_time': 'Int64',
        'end_time': 'Int64',
        'plan_cpu': 'float64',
        'plan_mem': 'float64'
    },
    names=bt_headers)
# dfs = []
# with pd.read_csv(batch_task_data_path, chunksize=chunk_size, iterator=True) as reader:
#     for chunk in reader:
#         df = pd.DataFrame(chunk)
#         dfs.append(df)
# batch_task_dataframe = pd.concat(dfs)

# read container_usage table
'''container_usage_data_path = 'data/container_usage.tar.gz'
cu_headers=['container_id','machine_id','time_stamp','cpu_util_percent','mem_util_percent','cpi','mem_gps','mpki','net_in','net_out','disk_io_percent']
container_usage_data_frame = pd.read_csv(container_usage_data_path, names=cu_headers, encoding='unicode_escape')'''

# read batch_instance
batch_instance_data_path = Path('E:/batch_instance/batch_instance.csv')
bi_headers=["instance_name", "task_name","job_name","task_type","status","start_time","end_time","machine_id","seq_no","total_seq_no" ,"cpu_avg","cpu_max","mem_avg","mem_max"]
bi_reader = pd.read_csv(batch_instance_data_path, names=bi_headers, encoding='unicode_escape', nrows=100)
bi_reader
# batch_instance_dataframe = pd.read_csv(batch_instance_data_path, names=bi_headers, encoding='unicode_escape', nrows="100")


,instance_name,task_name,job_name,task_type,status,start_time,end_time,machine_id,seq_no,total_seq_no,cpu_avg,cpu_max,mem_avg,mem_max
0,ins_74901673,task_LTg0MTUwNTA5Mjg4MDkwNjIzMA==,j_217,10,Terminated,673795,673797,m_2637,1,1,13,16,0.02,0.02
1,ins_815802872,M1,j_1527,1,Terminated,158478,158520,m_3430,1,1,3,19,0.13,0.18
2,ins_564677701,M1,j_2014,1,Terminated,372602,372616,m_1910,1,1,87,116,0.04,0.05
3,ins_257566161,M1,j_2014,1,Terminated,372602,372615,m_2485,1,1,91,123,0.05,0.05
4,ins_688679908,M1,j_2014,1,Terminated,372602,372615,m_993,1,1,93,141,0.05,0.05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,ins_1013997266,M1,j_2014,1,Terminated,372603,372619,m_686,1,1,75,97,0.04,0.05
96,ins_1077554435,M1,j_2014,1,Terminated,372602,372617,m_1884,1,1,80,132,0.05,0.05
97,ins_727049245,M1,j_2014,1,Terminated,372602,372613,m_516,1,1,96,164,0.05,0.05
98,ins_1077554826,M1,j_2014,1,Terminated,372603,372617,m_826,1,1,81,99,0.04,0.05


In [12]:
# pd.merge(batch_instance_dataframe, batch_task_dataframe, how="inner", on="")
tmp = pd.merge(machine_meta_dataframe, container_meta_dataframe, how="inner", on="machine_id", suffixes=["_m","_c"])
tmp[tmp["machine_id"]=="m_1"]
# result = pd.merge(tmp, batch_task_dataframe, how="inner", on="machine_id")
# result.head()

,machine_id,time_stamp_m,failure_domain_1,failure_domain_2,cpu_num,mem_size_m,status_m,container_id,time_stamp_c,app_du,status_c,cpu_request,cpu_limit,mem_size_c
0,m_1,0,219,17.0,96,100,USING,c_4801,0,app_3978,started,400,400,1.56
1,m_1,0,219,17.0,96,100,USING,c_4801,146012,app_3978,started,400,400,1.56
2,m_1,0,219,17.0,96,100,USING,c_4801,286480,app_3978,started,400,400,1.56
3,m_1,0,219,17.0,96,100,USING,c_4801,381777,app_3978,started,400,400,1.56
4,m_1,0,219,17.0,96,100,USING,c_11088,145633,app_3469,started,400,400,1.56
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315,m_1,683653,219,17.0,96,100,USING,c_58938,680720,app_4181,started,400,400,1.56
316,m_1,683653,219,17.0,96,100,USING,c_59922,0,app_9075,started,400,400,1.56
317,m_1,683653,219,17.0,96,100,USING,c_59922,146012,app_9075,started,400,400,1.56
318,m_1,683653,219,17.0,96,100,USING,c_59922,286480,app_9075,started,400,400,1.56
